In [3]:
import numpy as np

In [125]:
class BasicAnn:
    def __init__(self, layers):
        self._layers = layers
        self._num_layers = len(layers)
        self._parameters = self._set_parameters(layers)
        self._activations = {}
        self._linear_acts = {}
        self._gradients = {}
        
    def _set_parameters(self, layers):
        L = self._num_layers
        parameters = {}
        
        for l in range(1, L):
            parameters[f"W{l}"] = np.random.randn(layers[l][0], layers[l-1][0]) * 0.01
            parameters[f"b{l}"] = np.zeros((layers[l][0], 1)) * 0.01
        
        return parameters
    
    def _activation(self, x, activ_func):
        if activ_func == "sigmoid":
            return 1/(1+np.exp(-x))
        elif activ_func == "tanh":
            return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        elif activ_func == "relu":
            return x * (x > 0)
        elif activ_func == "linear":
            return x
    
    def _activation_deriv(self, a, activ_func):
        """
        Function to return value with the derivative of the selected activation
        """
        if activ_func == "sigmoid":
            return a * (1 - a)
        elif activ_func == "tanh":
            return 1 - a**2
        elif activ_func == "relu":
            return 1 * (a > 0)
        elif activ_func == "linear":
            return np.ones(a.shape)
    
    def train(self, X, Y, epochs=1, l_rate=0.01):
        for i in range(epochs):
            # forward pass
            self._forward_pass(X)
            output = self._activations[f"A{self._num_layers - 1}"]
            
            # error calculation
            cost = self._compute_cost(output, Y, "mean_square")
            print(f"Cost after epoch {i + 1}: {cost}")
            
            # backward pass
            self._compute_gradients(output, Y)
            self._update_parameters(l_rate)
        
    def _forward_pass(self, X):
        act = X
        self._activations["A0"] = act
        L = self._num_layers
        
        for l in range(1, L):
            W, b = self._parameters[f"W{l}"], self._parameters[f"b{l}"]
            Z = np.dot(W, act) + b
            act = self._activation(Z, self._layers[l][1])
            self._linear_acts[f"Z{l}"] = Z
            self._activations[f"A{l}"] = act
    
    def _compute_cost(self, AL, Y, cost_func="cross_entropy"):
        m = Y.shape[1]
        
        if cost_func == "cross_entropy":
            cost = -1/m * np.sum(
                np.multiply(np.log(AL), Y) + np.multiply((1 - Y), np.log(1 - AL))
            )
            cost = np.squeeze(cost)
        elif cost_func == "mean_square":
            cost = 1/m * np.sum((AL - Y) ** 2)
        
        return cost
    
    def _output_deriv(self, AL, Y, cost_func="cross_entropy"):
        """
        Outputs the derivate of the output activation layer with 
        respect to a given cost function
        """
        if cost_func == "cross_entropy":
            dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        elif cost_func == "mean_square":
            dAL = 2 * (AL - Y)
        
        return dAL
    
    def _compute_gradients(self, AL, Y):
        dAL = self._output_deriv(AL, Y)
        L = self._num_layers
        m = Y.shape[1]
        
        dZL = AL * self._activation_deriv(AL, self._layers[L-1][1])
        dWL = 1/m * np.dot(dZL, self._activations[f"A{L-2}"].T)
        dbL = 1/m * np.sum(dZL, axis=1, keepdims=True)
        
        self._gradients[f"dA{L - 1}"] = dAL
        self._gradients[f"dW{L - 1}"] = dWL
        self._gradients[f"db{L - 1}"] = dbL
        self._gradients[f"dA{L - 2}"] = np.dot(self._parameters[f"W{L - 1}"].T, dZL)
        
        for l in range(L - 2, 0, -1):
            A = self._activations[f"A{l}"]
            W = self._parameters[f"W{l}"]
            
            dZ = A * self._activation_deriv(A, self._layers[l][1])
            dW = 1/m * np.dot(dZ, self._activations[f"A{l-1}"].T)
            db = 1/m * np.sum(dZ, axis=1, keepdims=True)
            
            self._gradients[f"dA{l-1}"] = np.dot(W.T, dZ)
            self._gradients[f"dW{l}"] = dW
            self._gradients[f"db{l}"] = db

    def _update_parameters(self, l_rate):
        L = self._num_layers
        
        for l in range(1, L):
            self._parameters[f"W{l}"] -= l_rate * self._gradients[f"dW{l}"]
            self._parameters[f"b{l}"] -= l_rate * self._gradients[f"db{l}"]

In [126]:
X = np.random.randn(5,100)
Y = np.random.randn(1,100)

In [127]:
myAnn = BasicAnn(((5,"input"),(4, "relu"),(5, "tanh"), (2, "relu"), (1, "sigmoid")))
myAnn.train(X, Y, epochs=5000)

Cost after epoch 1: 1.2332980802011047
Cost after epoch 2: 1.2331027919272608
Cost after epoch 3: 1.2329078210654218
Cost after epoch 4: 1.2327131674806842
Cost after epoch 5: 1.2325188310186388
Cost after epoch 6: 1.23232481154118
Cost after epoch 7: 1.232131108898821
Cost after epoch 8: 1.2319377229384838
Cost after epoch 9: 1.2317446535224532
Cost after epoch 10: 1.2315519004936983
Cost after epoch 11: 1.2313594637110055
Cost after epoch 12: 1.2311673430011125
Cost after epoch 13: 1.2309755382160847
Cost after epoch 14: 1.2307840492032087
Cost after epoch 15: 1.2305928757955118
Cost after epoch 16: 1.2304020178291477
Cost after epoch 17: 1.2302114751492834
Cost after epoch 18: 1.2300212475917158
Cost after epoch 19: 1.2298313349808156
Cost after epoch 20: 1.2296417371438755
Cost after epoch 21: 1.2294524539096174
Cost after epoch 22: 1.229263485107823
Cost after epoch 23: 1.229074830555576
Cost after epoch 24: 1.2288864900836332
Cost after epoch 25: 1.2286984635122002
Cost after epo

Cost after epoch 501: 1.169399649220567
Cost after epoch 502: 1.169325262047481
Cost after epoch 503: 1.169251030351544
Cost after epoch 504: 1.1691769538251153
Cost after epoch 505: 1.169103032161011
Cost after epoch 506: 1.169029265052505
Cost after epoch 507: 1.168955652193327
Cost after epoch 508: 1.1688821932776667
Cost after epoch 509: 1.1688088880001706
Cost after epoch 510: 1.1687357360559447
Cost after epoch 511: 1.1686627371405534
Cost after epoch 512: 1.1685898909500214
Cost after epoch 513: 1.168517197180833
Cost after epoch 514: 1.1684446555299333
Cost after epoch 515: 1.1683722656947269
Cost after epoch 516: 1.1683000273730806
Cost after epoch 517: 1.1682279402633227
Cost after epoch 518: 1.1681560040642418
Cost after epoch 519: 1.1680842184750904
Cost after epoch 520: 1.1680125831955828
Cost after epoch 521: 1.167941097925895
Cost after epoch 522: 1.1678697623666676
Cost after epoch 523: 1.1677985762190037
Cost after epoch 524: 1.1677275391844701
Cost after epoch 525: 1.

Cost after epoch 1076: 1.1446011475838436
Cost after epoch 1077: 1.1445791148624544
Cost after epoch 1078: 1.1445571290261443
Cost after epoch 1079: 1.1445351899776355
Cost after epoch 1080: 1.1445132976198644
Cost after epoch 1081: 1.1444914518559814
Cost after epoch 1082: 1.1444696525893487
Cost after epoch 1083: 1.1444478997235403
Cost after epoch 1084: 1.1444261931623434
Cost after epoch 1085: 1.144404532809755
Cost after epoch 1086: 1.1443829185699839
Cost after epoch 1087: 1.1443613503474486
Cost after epoch 1088: 1.1443398280467776
Cost after epoch 1089: 1.1443183515728095
Cost after epoch 1090: 1.1442969208305918
Cost after epoch 1091: 1.1442755357253793
Cost after epoch 1092: 1.144254196162636
Cost after epoch 1093: 1.1442329020480337
Cost after epoch 1094: 1.1442116532874504
Cost after epoch 1095: 1.1441904497869717
Cost after epoch 1096: 1.1441692914528883
Cost after epoch 1097: 1.1441481781916985
Cost after epoch 1098: 1.144127109910104
Cost after epoch 1099: 1.144106086515

Cost after epoch 1517: 1.1383291416329484
Cost after epoch 1518: 1.1383207637839499
Cost after epoch 1519: 1.1383124052472802
Cost after epoch 1520: 1.1383040659854913
Cost after epoch 1521: 1.138295745961212
Cost after epoch 1522: 1.1382874451371514
Cost after epoch 1523: 1.1382791634760954
Cost after epoch 1524: 1.1382709009409084
Cost after epoch 1525: 1.1382626574945334
Cost after epoch 1526: 1.1382544330999906
Cost after epoch 1527: 1.138246227720378
Cost after epoch 1528: 1.138238041318871
Cost after epoch 1529: 1.1382298738587227
Cost after epoch 1530: 1.1382217253032627
Cost after epoch 1531: 1.1382135956158979
Cost after epoch 1532: 1.1382054847601115
Cost after epoch 1533: 1.1381973926994637
Cost after epoch 1534: 1.1381893193975914
Cost after epoch 1535: 1.1381812648182068
Cost after epoch 1536: 1.138173228925099
Cost after epoch 1537: 1.138165211682132
Cost after epoch 1538: 1.1381572130532471
Cost after epoch 1539: 1.1381492330024596
Cost after epoch 1540: 1.13814127149386

Cost after epoch 2085: 1.1358099220095437
Cost after epoch 2086: 1.1358082738944402
Cost after epoch 2087: 1.1358066324841896
Cost after epoch 2088: 1.135804997766746
Cost after epoch 2089: 1.1358033697300858
Cost after epoch 2090: 1.135801748362208
Cost after epoch 2091: 1.1358001336511345
Cost after epoch 2092: 1.13579852558491
Cost after epoch 2093: 1.1357969241516006
Cost after epoch 2094: 1.1357953293392968
Cost after epoch 2095: 1.13579374113611
Cost after epoch 2096: 1.1357921595301748
Cost after epoch 2097: 1.1357905845096476
Cost after epoch 2098: 1.1357890160627073
Cost after epoch 2099: 1.1357874541775554
Cost after epoch 2100: 1.1357858988424159
Cost after epoch 2101: 1.1357843500455342
Cost after epoch 2102: 1.135782807775178
Cost after epoch 2103: 1.1357812720196367
Cost after epoch 2104: 1.1357797427672238
Cost after epoch 2105: 1.1357782200062714
Cost after epoch 2106: 1.135776703725137
Cost after epoch 2107: 1.1357751939121976
Cost after epoch 2108: 1.135773690555853
C

Cost after epoch 2560: 1.1356077784259393
Cost after epoch 2561: 1.1356082813590438
Cost after epoch 2562: 1.1356087871779048
Cost after epoch 2563: 1.1356092958774224
Cost after epoch 2564: 1.1356098074525047
Cost after epoch 2565: 1.1356103218980698
Cost after epoch 2566: 1.1356108392090432
Cost after epoch 2567: 1.1356113593803607
Cost after epoch 2568: 1.1356118824069656
Cost after epoch 2569: 1.1356124082838102
Cost after epoch 2570: 1.1356129370058563
Cost after epoch 2571: 1.1356134685680739
Cost after epoch 2572: 1.1356140029654413
Cost after epoch 2573: 1.1356145401929463
Cost after epoch 2574: 1.1356150802455849
Cost after epoch 2575: 1.1356156231183616
Cost after epoch 2576: 1.13561616880629
Cost after epoch 2577: 1.1356167173043923
Cost after epoch 2578: 1.135617268607699
Cost after epoch 2579: 1.135617822711249
Cost after epoch 2580: 1.1356183796100905
Cost after epoch 2581: 1.1356189392992797
Cost after epoch 2582: 1.1356195017738815
Cost after epoch 2583: 1.1356200670289

Cost after epoch 3036: 1.136098116000973
Cost after epoch 3037: 1.1360995443960824
Cost after epoch 3038: 1.1361009740099384
Cost after epoch 3039: 1.136102404840244
Cost after epoch 3040: 1.1361038368847043
Cost after epoch 3041: 1.136105270141029
Cost after epoch 3042: 1.136106704606932
Cost after epoch 3043: 1.136108140280129
Cost after epoch 3044: 1.1361095771583418
Cost after epoch 3045: 1.1361110152392937
Cost after epoch 3046: 1.136112454520714
Cost after epoch 3047: 1.1361138950003333
Cost after epoch 3048: 1.1361153366758878
Cost after epoch 3049: 1.1361167795451157
Cost after epoch 3050: 1.1361182236057605
Cost after epoch 3051: 1.136119668855568
Cost after epoch 3052: 1.1361211152922883
Cost after epoch 3053: 1.1361225629136749
Cost after epoch 3054: 1.1361240117174856
Cost after epoch 3055: 1.1361254617014807
Cost after epoch 3056: 1.1361269128634253
Cost after epoch 3057: 1.136128365201087
Cost after epoch 3058: 1.136129818712238
Cost after epoch 3059: 1.1361312733946534
C

Cost after epoch 3591: 1.137025314489836
Cost after epoch 3592: 1.1370271507126741
Cost after epoch 3593: 1.1370289873072175
Cost after epoch 3594: 1.1370308242725093
Cost after epoch 3595: 1.1370326616075948
Cost after epoch 3596: 1.1370344993115216
Cost after epoch 3597: 1.1370363373833376
Cost after epoch 3598: 1.1370381758220927
Cost after epoch 3599: 1.137040014626839
Cost after epoch 3600: 1.1370418537966283
Cost after epoch 3601: 1.1370436933305152
Cost after epoch 3602: 1.1370455332275557
Cost after epoch 3603: 1.1370473734868067
Cost after epoch 3604: 1.1370492141073272
Cost after epoch 3605: 1.1370510550881776
Cost after epoch 3606: 1.1370528964284188
Cost after epoch 3607: 1.1370547381271139
Cost after epoch 3608: 1.1370565801833277
Cost after epoch 3609: 1.1370584225961262
Cost after epoch 3610: 1.1370602653645763
Cost after epoch 3611: 1.1370621084877475
Cost after epoch 3612: 1.137063951964709
Cost after epoch 3613: 1.1370657957945338
Cost after epoch 3614: 1.137067639976

Cost after epoch 4095: 1.1379809949822732
Cost after epoch 4096: 1.1379829234375876
Cost after epoch 4097: 1.1379848519295355
Cost after epoch 4098: 1.1379867804576789
Cost after epoch 4099: 1.1379887090215788
Cost after epoch 4100: 1.137990637620798
Cost after epoch 4101: 1.1379925662548993
Cost after epoch 4102: 1.1379944949234468
Cost after epoch 4103: 1.1379964236260047
Cost after epoch 4104: 1.1379983523621386
Cost after epoch 4105: 1.1380002811314136
Cost after epoch 4106: 1.1380022099333966
Cost after epoch 4107: 1.1380041387676545
Cost after epoch 4108: 1.1380060676337556
Cost after epoch 4109: 1.138007996531268
Cost after epoch 4110: 1.1380099254597607
Cost after epoch 4111: 1.1380118544188043
Cost after epoch 4112: 1.1380137834079684
Cost after epoch 4113: 1.1380157124268246
Cost after epoch 4114: 1.138017641474945
Cost after epoch 4115: 1.138019570551901
Cost after epoch 4116: 1.1380214996572673
Cost after epoch 4117: 1.1380234287906164
Cost after epoch 4118: 1.1380253579515

Cost after epoch 4600: 1.138951744098176
Cost after epoch 4601: 1.1389536473813162
Cost after epoch 4602: 1.138955550548088
Cost after epoch 4603: 1.1389574535982936
Cost after epoch 4604: 1.1389593565317382
Cost after epoch 4605: 1.138961259348226
Cost after epoch 4606: 1.1389631620475624
Cost after epoch 4607: 1.1389650646295517
Cost after epoch 4608: 1.1389669670940006
Cost after epoch 4609: 1.1389688694407136
Cost after epoch 4610: 1.138970771669498
Cost after epoch 4611: 1.1389726737801598
Cost after epoch 4612: 1.1389745757725056
Cost after epoch 4613: 1.138976477646343
Cost after epoch 4614: 1.1389783794014792
Cost after epoch 4615: 1.1389802810377219
Cost after epoch 4616: 1.1389821825548796
Cost after epoch 4617: 1.1389840839527603
Cost after epoch 4618: 1.1389859852311728
Cost after epoch 4619: 1.1389878863899265
Cost after epoch 4620: 1.1389897874288302
Cost after epoch 4621: 1.138991688347694
Cost after epoch 4622: 1.1389935891463279
Cost after epoch 4623: 1.138995489824542